In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9810569973762879850, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 16185556992
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 12238099409443257790
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [3]:
!pip install imgaug --upgrade

     |████████████████████████████████| 948 kB 5.1 MB/s 
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.4.0 which is incompatible.


In [4]:
# GPU 환경 설정
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

# 파일경로 설정
import json

# 데이터 보기
import pandas as pd
import numpy as np
from glob import glob

# 이미지데이터 로딩
import cv2
from tqdm import tqdm

# Others
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')

# Split valid-set from train-set

In [ ]:
src_path = '/gdrive/MyDrive/dacon/handSignal/train'


In [26]:
'''Generator'''
from tensorflow.keras.utils import Sequence
from google.colab.patches import cv2_imshow
from imgaug import augmenters as iaa
from imgaug.augmentables.batches import UnnormalizedBatch
import random

class Generator(Sequence):
    def __init__(self, src_path, input_shape, batch_size, augs=None, split_ratio=0.9, is_train=True):
        
        new_image_directory = src_path + '/new_images'
        new_train_image_directory = new_image_directory + '/train'

        action_information = pd.read_csv(src_path + '/action_information.csv')

        classes = pd.get_dummies(action_information[['Label']], columns = ['Label']).to_numpy()
        
        new_train_image_directories = sorted(glob(new_train_image_directory + '/*'), key = lambda x : int(x.split('/')[-1].split('_')[-1]))

        train_answer = []
        train_image_directories = sorted(glob(new_train_image_directory + '/*'), key = lambda x : int(x.split('_')[-1]))
        for train_image_directory in train_image_directories : 
            json_path = glob(train_image_directory + '/*.json')[0]
            js = json.load(open(json_path))
            action = js.get('action')
            train_answer.append(action)
        
        self.class_num = len(classes)
        self.is_train = is_train
        self.x_data = []
        self.y_data = []
        self.batch_size = batch_size
        self.input_shape = input_shape
        self.augs = iaa.Sequential(augs)

        for new_train_image_directory, action in tqdm(zip(new_train_image_directories, train_answer), total = len(new_train_image_directories)) : 
            image_paths = sorted(glob(new_train_image_directory + '/*.jpg'), key = lambda x : int(x.split('/')[-1].replace('.jpg','')))
            image_len = len(image_paths)

            random.seed(777)
            random.shuffle(image_paths)

            if is_train:
                self.x_data +=image_paths[:round(image_len*split_ratio)]
            else:
                self.x_data +=image_paths[round(image_len*split_ratio):]
            
            for i in range(len(self.x_data)):
                self.y_data.append(classes[action])

        print(len(self.x_data))
        self.on_epoch_end()
        print("Generator Initialized!!")
    
    def __len__(self):
        return round(len(self.x_data) / self.batch_size)
    
    def __getitem__(self, idx):
        batch_x, batch_y = [], []
        batch_index = self.index[idx * self.batch_size:(idx+1)*self.batch_size]
        
        for batch_i in batch_index:
            batch_x.append(self.x_data[batch_i])
            batch_y.append(self.y_data[batch_i])
        
        out_x, out_y = self.data_gen(batch_x, batch_y)
        return out_x, out_y

    def on_epoch_end(self):
        self.index = np.arange(len(self.x_data))
        if self.is_train:
            np.random.shuffle(self.index)
    
    def data_gen(self, x ,y):
        input_x = np.zeros((self.batch_size, self.input_shape[0], self.input_shape[1], 3), dtype=np.float)
        input_y = np.zeros((self.batch_size, self.class_num), dtype=np.float)

        imgs = []
        for i in range(len(x)):
            img = cv2.imread(x[i])
            imgs.append(cv2.resize(img, (self.input_shape[0], self.input_shape[1])))
        
        batch_imgs = UnnormalizedBatch(images=imgs, data=y)
        batch_aug_imgs = list(self.augs.augment_batches(batches=batch_imgs))

        for i in range(len(x)):
            aug_img = batch_aug_imgs[0].images_aug[i]
            # cv2_imshow(aug_img)
            # cv2_imshow(batch_aug_imgs[0].images_unaug[i])
            input_x[i]= aug_img.astype(np.float) / 255.0
            input_y[i] = y[i]
        return input_x, input_y
        


In [10]:
from tensorflow.keras.applications import EfficientNetB7 # 모델은 가벼운 모델을 사용합니다.
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

def buildModel():
    baseModel = EfficientNetB7(input_shape = (224,224,3), weights='imagenet', include_top=False, )
    baseModel.trainable = True
    model_in = Input(shape = (224,224,3))
    base_model = baseModel(model_in)
    head_model = MaxPooling2D(pool_size=(7, 7))(base_model)
    head_model = Flatten(name="flatten")(head_model)
    head_model = Dense(256, activation = 'relu')(head_model)
    head_model = Dropout(0.2)(head_model)
    head_model = Dense(32, activation = 'relu')(head_model)
    head_model = Dropout(0.2)(head_model)
    head_model = Dense(8, activation = 'relu')(head_model)
    head_model = Dropout(0.2)(head_model)
    model_out = Dense(6, activation="softmax")(head_model)

    model = Model(inputs=model_in, outputs=model_out)
    return model

In [11]:
import imgaug.augmenters as iaa

augs =[
    #    iaa.Fliplr(0.5),
       
        # iaa.Sometimes(0.2,iaa.Sharpen()),
       
        iaa.SomeOf((0,2),[
            iaa.MultiplyAndAddToBrightness(),
            iaa.GammaContrast()
        ]),

        # iaa.SomeOf((0,1), [
        #     iaa.Sometimes(0.7, iaa.AdditiveGaussianNoise()),
        # ]),

        iaa.SomeOf((0,2),[
            iaa.ScaleX((0.8, 1.2)),
            iaa.ScaleY((0.8, 1.2)),
        ]),

]

In [ ]:
# Modeling
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

'''Train options'''
src_path = '/gdrive/MyDrive/dacon/handSignal'
batch_size = 16
input_shape = (224, 224)
epochs = 2000
lr = 1e-3
model_name = "effient"
filename = src_path+"/save_weights/%s_{epoch:05d}.h5"%(model_name)

callback = [ReduceLROnPlateau(monitor='loss', factor=0.1, patience=5),
            ModelCheckpoint(filename, monitor="loss", save_best_only=True)]

'''Build Model'''
with tf.device('/device:GPU:0'):
    model = buildModel()
    # model.load_weights(src_path+"/save_weights/effient_00039.h5")
    model.compile(loss='categorical_crossentropy',optimizer=SGD(learning_rate=lr, momentum=0.9), metrics=['accuracy'])

    '''Train'''
    train_gen = Generator(src_path, input_shape, batch_size, augs=augs, is_train=True)
    val_gen = Generator(src_path, input_shape, batch_size, augs=[], is_train=False)
    iteration = round(len(train_gen) / batch_size)
    model.fit_generator(train_gen, validation_data=val_gen, validation_steps=iteration,epochs=epochs, max_queue_size=50, workers=32, callbacks=callback)

100%|██████████| 142/142 [00:00<00:00, 160.71it/s]


19993
Generator Initialized!!


100%|██████████| 142/142 [00:00<00:00, 326.79it/s]


2225
Generator Initialized!!
Epoch 1/2000
 150/1250 [==>...........................] - ETA: 11:34 - loss: 1.7188 - accuracy: 0.4996

In [ ]:
!nvidia-smi

In [24]:
'''for test generator'''
src_path = '/gdrive/MyDrive/dacon/handSignal'
batch_size = 1
input_shape = (224, 224)
augs= [
       iaa.Fliplr(1.0)
]
gen = Generator(src_path, input_shape, batch_size, augs=augs, is_train=True)
gen.__getitem__(0)

  0%|          | 0/142 [00:00<?, ?it/s]

['/gdrive/MyDrive/dacon/handSignal/new_images/train/file_0/30.jpg', '/gdrive/MyDrive/dacon/handSignal/new_images/train/file_0/36.jpg', '/gdrive/MyDrive/dacon/handSignal/new_images/train/file_0/62.jpg', '/gdrive/MyDrive/dacon/handSignal/new_images/train/file_0/83.jpg', '/gdrive/MyDrive/dacon/handSignal/new_images/train/file_0/150.jpg', '/gdrive/MyDrive/dacon/handSignal/new_images/train/file_0/143.jpg', '/gdrive/MyDrive/dacon/handSignal/new_images/train/file_0/84.jpg', '/gdrive/MyDrive/dacon/handSignal/new_images/train/file_0/98.jpg', '/gdrive/MyDrive/dacon/handSignal/new_images/train/file_0/114.jpg', '/gdrive/MyDrive/dacon/handSignal/new_images/train/file_0/142.jpg', '/gdrive/MyDrive/dacon/handSignal/new_images/train/file_0/58.jpg', '/gdrive/MyDrive/dacon/handSignal/new_images/train/file_0/41.jpg', '/gdrive/MyDrive/dacon/handSignal/new_images/train/file_0/24.jpg', '/gdrive/MyDrive/dacon/handSignal/new_images/train/file_0/10.jpg', '/gdrive/MyDrive/dacon/handSignal/new_images/train/file_0

TypeError: ignored

# Prediction

In [ ]:
src_path = '/gdrive/MyDrive/dacon/handSignal'

new_test_image_directory = src_path + '/new_images/test'
new_test_image_directories = sorted(glob(new_test_image_directory + '/*'), key = lambda x : int(x.split('file_')[-1]))
model = buildModel()
model.load_weights(src_path+"/save_weights/effient_00022.h5")

predictions = []
for new_test_image_directory in tqdm(new_test_image_directories, total = len(new_test_image_directories)) :
    image_paths = sorted(glob(new_test_image_directory + '/*.jpg'), key = lambda x : int(x.split('/')[-1].replace('.jpg','')))
    image_len = len(image_paths)
    test_images  = []
    for image_path in image_paths:
        img = cv2.imread(image_path)
        img = cv2.resize(img, (224, 224))
        img = img/255
        test_images.append(img)
    prediction = np.mean(model.predict(np.array(test_images)), axis = 0)
    predictions.append(prediction)

100%|██████████| 45/45 [32:07<00:00, 42.84s/it]


In [ ]:
sample_submission = pd.read_csv(src_path + '/sample_submission.csv')
sample_submission.iloc[:,1:] = predictions
display(sample_submission.head())
sample_submission.to_csv(src_path+'/effcientB0_3.csv', index=False)

,file_path,Label_0,Label_1,Label_2,Label_3,Label_4,Label_5
0,./test\file_142,1.175328e-02,7.120891e-03,4.155934e-01,5.422419e-01,3.936142e-03,1.935453e-02
1,./test\file_143,9.483405e-10,1.015056e-15,5.341049e-04,3.173556e-13,1.380441e-16,9.994661e-01
2,./test\file_144,1.887312e-03,9.407359e-04,1.304676e-01,8.606133e-01,5.755061e-04,5.515538e-03
3,./test\file_145,1.530295e-03,1.113463e-04,8.130416e-01,8.722627e-04,2.200164e-05,1.844224e-01
4,./test\file_146,9.981704e-01,1.829489e-03,1.021436e-10,6.640961e-09,1.677697e-07,1.175347e-09
